In [1]:
from utils import *
import os.path
from scipy.stats import qmc
from tqdm import tqdm
from pymoo.operators.sampling.lhs import LHS
import polars as pl

In [2]:
def norm_y(df):
    df = df.with_columns([
        (pl.col('y') - pl.col('y').min()).over(['problem', 'instance']).alias('y_min_max_norm')
    ]).with_columns([
        (pl.col('y_min_max_norm') / pl.col('y_min_max_norm').max()).over(['problem', 'instance']).alias('y')
    ]).drop('y_min_max_norm')
    return df

In [3]:
save_dir = 'lio_problem'
create_directory_if_not_exist(save_dir)

In [4]:
sample_size = 250 #times dimention
problem_dim = 5

In [5]:
file = f'{save_dir}/samples__dim_{problem_dim}__samplesize_{sample_size}.parquet'

In [6]:
df = pl.scan_parquet(file).collect()

In [7]:
df

problem,instance,y,optimum,x_0,x_1,x_2,x_3,x_4
i64,i64,f64,f64,f64,f64,f64,f64,f64
1,1,183.443152,79.48,-3.149596,1.601646,4.805492,-2.859105,2.913973
1,1,137.493662,79.48,0.78446,1.718528,3.697858,2.556515,2.751843
1,1,111.218935,79.48,-0.651181,-0.647084,-1.350508,-3.267599,-4.495132
1,1,162.434749,79.48,-3.057518,-2.772905,4.3904,-4.642849,-2.411301
1,1,101.772582,79.48,1.708115,1.237669,-0.079004,2.538148,1.013913
…,…,…,…,…,…,…,…,…
24,15,373.042109,149.81,3.550095,-2.349579,0.610341,-4.475076,-4.671802
24,15,307.514824,149.81,-2.14731,-4.21336,-0.549176,-1.346791,-0.135708
24,15,314.330617,149.81,-4.200217,2.243036,-3.145434,-0.55592,-1.302764


In [8]:
save_file = f'{save_dir}/samples__dim_{problem_dim}__samplesize_{sample_size}.norm.parquet'

In [9]:
df = norm_y(df.lazy()).collect()
df

problem,instance,y,optimum,x_0,x_1,x_2,x_3,x_4
i64,i64,f64,f64,f64,f64,f64,f64,f64
1,1,0.577262,79.48,-3.149596,1.601646,4.805492,-2.859105,2.913973
1,1,0.318008,79.48,0.78446,1.718528,3.697858,2.556515,2.751843
1,1,0.169762,79.48,-0.651181,-0.647084,-1.350508,-3.267599,-4.495132
1,1,0.458729,79.48,-3.057518,-2.772905,4.3904,-4.642849,-2.411301
1,1,0.116464,79.48,1.708115,1.237669,-0.079004,2.538148,1.013913
…,…,…,…,…,…,…,…,…
24,15,0.705769,149.81,3.550095,-2.349579,0.610341,-4.475076,-4.671802
24,15,0.460143,149.81,-2.14731,-4.21336,-0.549176,-1.346791,-0.135708
24,15,0.485692,149.81,-4.200217,2.243036,-3.145434,-0.55592,-1.302764


In [10]:
df.write_parquet(save_file)